In [19]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
import os
import warnings
warnings.filterwarnings('ignore')
os.environ["PYTHONWARNINGS"] = "ignore"

In [21]:
cur_folder_name = os.getcwd().split('/')[-1]
if cur_folder_name != "Virny":
    os.chdir("../..")

print('Current location: ', os.getcwd())

Current location:  /Users/denys_herasymuk/UCU/4course_2term/Bachelor_Thesis/Code/Virny


# Multiple Models Interface Usage

In [22]:
import os
import pandas as pd

from virny.datasets import ACSPublicCoverageDataset
from virny.custom_classes.metrics_composer import MetricsComposer
from virny.custom_classes.metrics_interactive_visualizer import MetricsInteractiveVisualizer

In [23]:
data_loader = ACSPublicCoverageDataset(state=['CA'], year=2018, with_nulls=False,
                                       subsample_size=15_000, subsample_seed=42)
sensitive_attributes_dct = {'SEX': '2', 'RAC1P': ['2', '3', '4', '5', '6', '7', '8', '9'], 'SEX&RAC1P': None}

In [24]:
ROOT_DIR = os.path.join('docs', 'examples')
subgroup_metrics_df = pd.read_csv(os.path.join(ROOT_DIR, 'pub_cov_subgroup_metrics.csv'), header=0)
subgroup_metrics_df['Model_Name'] = (subgroup_metrics_df['Model_Name'] + '__alpha=' +
                                     subgroup_metrics_df['Intervention_Param'].astype(str))

In [25]:
model_names = subgroup_metrics_df['Model_Name'].unique()
models_metrics_dct = dict()
for model_name in model_names:
    models_metrics_dct[model_name] = subgroup_metrics_df[subgroup_metrics_df['Model_Name'] == model_name]

metrics_composer = MetricsComposer(models_metrics_dct, sensitive_attributes_dct)
models_composed_metrics_df = metrics_composer.compose_metrics()
models_composed_metrics_df.head()

,Metric,SEX,RAC1P,SEX&RAC1P,Model_Name
0,Accuracy_Parity,0.026847,0.016299,0.040212,LGBMClassifier__alpha=0.6
1,Aleatoric_Uncertainty_Parity,-0.013240,0.027276,0.007235,LGBMClassifier__alpha=0.6
2,Aleatoric_Uncertainty_Ratio,0.983584,1.034689,1.009077,LGBMClassifier__alpha=0.6
3,Equalized_Odds_FNR,0.004275,-0.000359,-0.008617,LGBMClassifier__alpha=0.6
4,Equalized_Odds_FPR,-0.012072,-0.024172,-0.040481,LGBMClassifier__alpha=0.6


In [26]:
models_metrics_dct.keys()

dict_keys(['LGBMClassifier__alpha=0.6', 'LGBMClassifier__alpha=0.0', 'LogisticRegression__alpha=0.0', 'LogisticRegression__alpha=0.6', 'MLPClassifier__alpha=0.0', 'MLPClassifier__alpha=0.6', 'RandomForestClassifier__alpha=0.0', 'RandomForestClassifier__alpha=0.6'])

## Metrics Visualization and Reporting

In [27]:
visualizer = MetricsInteractiveVisualizer(data_loader.X_data, data_loader.y_data,
                                          models_metrics_dct, models_composed_metrics_df,
                                          sensitive_attributes_dct=sensitive_attributes_dct)

In [28]:
visualizer.start_web_app()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
Keyboard interruption in main thread... closing server.


In [29]:
visualizer.stop_web_app()

Closing server running on port: 7860
